<a href="https://colab.research.google.com/github/SorayaFrancoLopez/RN/blob/main/clasificar_emails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
df = pd.read_csv("/content/drive/MyDrive/e-mails.csv", delimiter=';', encoding='latin1')

In [30]:
df

,emails,asunto,texto,prioridad
0,cliente1@email.com,Solicitud de Préstamo,"Hola, necesito solicitar un préstamo para una ...",Alta
1,cliente2@email.com,Consulta de Inversiones,"Estimado Gestor, ¿cuáles son las opciones de i...",Media
2,cliente3@email.com,Reclamo sobre Cargo,"He notado un cargo incorrecto en mi cuenta, ¿p...",Alta
3,cliente4@email.com,Pregunta sobre Cuenta de Ahorro,Quiero saber cuál es la tasa de interés actual...,Baja
4,cliente5@email.com,Actualización de Contacto,"Estimado Gestor, he cambiado mi dirección y nú...",Media
...,...,...,...,...
95,cliente96@email.com,Solicitud de Tarjeta de Crédito Empresarial,Quiero solicitar una tarjeta de crédito empres...,Alta
96,cliente97@email.com,Problema con Transferencia Bancaria,He tenido un problema con una transferencia ba...,Alta
97,cliente98@email.com,Informe de Estado de Cuenta,¿Puedes proporcionarme una copia de mi estado ...,Media
98,cliente99@email.com,Pregunta sobre Hipoteca,"Estimado Gestor, necesito información sobre la...",Baja


In [31]:
# Preprocesamiento de texto (limpieza, tokenización, eliminación de stopwords, etc.)
vectorizer = TfidfVectorizer(max_features=100)
X = vectorizer.fit_transform(df['texto'])

In [32]:
#Codificado de prioridad
encoder = LabelEncoder()
y = encoder.fit_transform(df['prioridad'])

In [33]:
#division en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
model = keras.Sequential([
    keras.layers.Dense(64, input_shape=(100,), activation='relu'),  # Cambia 5000 a 100
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

In [35]:
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Preprocesamiento de texto (limpieza, tokenización, eliminación de stopwords, etc.)
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['texto'])

# Convertir la matriz dispersa de TF-IDF en una matriz densa
X_dense = X.toarray()

In [37]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [38]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

InvalidArgumentError: ignored

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Precisión en los datos de prueba: {test_acc}")
